<a href="https://colab.research.google.com/github/caldi99/Computer-Vision-Project/blob/main/ObjectDectection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fatal: not a git repository (or any of the parent directories): .git
